In [29]:
import pandas as pd
import numpy as np

In [30]:
from tqdm import tqdm

In [31]:
import collections

# Pre-processing BI data

In [32]:
path = './data/MosIndex_All.csv'
data = pd.read_csv(path, low_memory = False)

In [162]:
data.head()

,Date,County,Town,Village,VillageID,VillageLon,VillageLat,AreaType,HouseHold,InspectType,...,HIAeg,HILv,HILvAeg,CI,CILv,LI,LILv,Pupa,AI,Con100HH
0,2010/01/01,台北市,中正區,三愛里,6300500-031,121.530593083676,25.0358286563398,住宅,50,衛生所,...,0.000,0,0,0.000,0,0.000,0,0,0.0000,0.00
1,2010/01/01,嘉義市,西區,保安里,1002002-011,120.431350323348,23.4910599871089,住宅,156,衛生局,...,0.000,0,0,0.000,0,0.000,0,0,0.0000,0.00
2,2010/01/02,高雄市,鳳山區,生明里,6401200-045,120.36306711902,22.6176350677023,住宅,60,監測,...,3.333,1,1,5.714,3,0.000,0,0,0.0333,3.33
3,2010/01/02,台中市,大里區,塗城里,6602800-016,120.717131600963,24.0852343263972,住宅,50,衛生所,...,0.000,0,0,0.000,0,0.000,0,0,0.0000,0.00
4,2010/01/03,宜蘭縣,蘇澳鎮,存仁里,1000203-004,121.845912904116,24.6259386590972,住宅,60,衛生所,...,0.000,0,0,0.000,0,0.000,0,0,0.0000,0.00


In [509]:
data.describe()

,HouseHold,PosHH,PosHHAeg,ConIn,ConOut,ConAll,PosConIn,PosConOut,PosConAll,FAegIn,FAegOut,FAlbIn,FAlbOut,LarvaAeg,LarvaAlb,LarvaNEC,Pupa,AI,Con100HH
count,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000,444878.000000
mean,65.109315,1.326669,0.507595,5.638148,20.452254,26.172171,0.184651,1.488954,1.674230,0.049535,0.067731,0.004089,0.115299,6.854693,9.266107,3.959998,0.421498,0.002989,2.326221
std,123.080350,3.145880,2.321932,14.792145,28.104680,36.568270,1.111730,4.106503,4.642644,0.829089,0.911957,0.190068,1.485530,41.033189,62.603743,36.446042,6.376623,0.028172,4.587215
min,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,0.000000,0.000000,0.000000,7.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,50.000000,0.000000,0.000000,0.000000,13.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,55.000000,2.000000,0.000000,6.000000,24.000000,31.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,74006.000000,161.000000,200.000000,1162.000000,2068.000000,2068.000000,132.000000,284.000000,284.000000,153.000000,200.000000,64.000000,200.000000,10095.000000,7890.000000,5493.000000,999.000000,4.000000,248.330000


In [354]:
county = data['County'].unique().tolist()
len(county)

22

In [355]:
county.remove('新竹市')
county.remove('嘉義市')
county.remove('連江縣')
county.remove('金門縣')


In [356]:
print(county)

['台北市', '高雄市', '台中市', '宜蘭縣', '台南市', '新北市', '彰化縣', '花蓮縣', '雲林縣', '新竹縣', '嘉義縣', '桃園市', '澎湖縣', '南投縣', '屏東縣', '台東縣', '基隆市', '苗栗縣']


In [357]:
use_cols = ['Date', 'County', 'BI', 'HI', 'CI']
bi_hi_ci = ['Date', 'BI', 'HI', 'CI']

In [358]:
data_distill = data[use_cols]

In [365]:
data_distill.head()

,Date,County,BI,HI,CI
0,2010/01/01,台北市,0.000,0.000,0.000
1,2010/01/01,嘉義市,0.000,0.000,0.000
2,2010/01/02,高雄市,3.333,3.333,5.714
3,2010/01/02,台中市,0.000,0.000,0.000
4,2010/01/03,宜蘭縣,0.000,0.000,0.000


In [510]:
data_distill.describe()

,Date,County,BI,HI,CI
count,444878,444878,444878,444878,444878
unique,3448,22,5386,4570,4765
top,2018/08/01,台南市,0.000,0.000,0.000
freq,1036,134594,237545,239781,227009


In [362]:
# split data for 20 county

cnty_data = collections.defaultdict(list)

for i in tqdm(range(data_distill.shape[0])):
    
    row = data_distill.iloc[i, :].tolist()
    cnty = row[1]
    if cnty in county:
        cnty_data[cnty].append(row)
        

100%|██████████| 444878/444878 [00:55<00:00, 8002.03it/s]


In [562]:
# convert data each county from list to dataframe
def fill_none(county_df):


    county_df = county_df.replace(to_replace='None', value= float(0))
    county_df = county_df.replace(to_replace='NaN', value= float(0))

    county_df['BI'] = county_df['BI'].astype(float)
    county_df['CI'] = county_df['CI'].astype(float)
    county_df['HI'] = county_df['HI'].astype(float)

    return county_df

data_fin = collections.defaultdict()


for k, v in cnty_data.items():
    print('\n', k)   

    x = pd.DataFrame(cnty_data[k], columns = ['Date', 'County', 'BI', 'HI', 'CI'])
    xx = fill_none(x)
    print(xx.describe())
    # max CI
    temp = x['CI']
    temp3 = []
    temp5 = list(temp)
    print(len(temp5))
    for n in temp5:
        if n == 'None':
            continue
        else:
            temp3.append(float(n))

    print("Max CI: {}".format(np.max(temp3)))

    data_fin[k] = xx


 花蓮縣
                BI          HI           CI
count  9993.000000  9993.00000  9993.000000
mean      0.361301     0.30839     1.773202
std       1.458427     1.05499     8.654685
min       0.000000     0.00000     0.000000
25%       0.000000     0.00000     0.000000
50%       0.000000     0.00000     0.000000
75%       0.000000     0.00000     0.000000
max      73.684000    36.84200   100.000000
9993
Max CI: 100.0

 屏東縣
                 BI            HI            CI
count  29887.000000  29887.000000  29887.000000
mean       3.783418      2.775930      6.079814
std        6.770321      3.858123      7.812628
min        0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000
50%        1.886000      1.851000      3.703000
75%        4.854000      3.846000      9.090000
max      248.333000    100.000000    100.000000
29887
Max CI: 100.0

 桃園市
                 BI            HI            CI
count  14278.000000  14278.000000  14278.000000
mean       0.815406

In [547]:
for ct, df in data_fin.items():
    print(ct, '\n', df.describe(), '\n')


花蓮縣 
               Date County     BI     HI     CI
count         9993   9993   9993   9993   9993
unique        2513      1     35     28    116
top     2015/01/30    花蓮縣  0.000  0.000  0.000
freq            38   9993   8863   8866   7016 

屏東縣 
               Date County     BI     HI     CI
count        29887  29887  29887  29887  29887
unique        2623      1   2658   2279   2521
top     2011/11/16    屏東縣  0.000  0.000  0.000
freq            41  29887  10581  10581  10581 

桃園市 
               Date County     BI     HI     CI
count        14278  14278  14278  14278  14278
unique        2236      1    345    297    498
top     2012/10/02    桃園市  0.000  0.000  0.000
freq           300  14278  10588  10597  10139 

台北市 
               Date County     BI     HI     CI
count        16284  16284  16284  16284  16284
unique        2442      1    584    520    557
top     2011/10/21    台北市  0.000  0.000  0.000
freq            34  16284   9045   9495   9008 

嘉義縣 
               Date Cou

### Utils functions for BI

In [548]:
def bi_groupby_date(df):

    def mean_customized(x):
        #purpose: to calculate mean of a grouped column with mixed values, ex: [3, 6, 8, 'NaN', 'T', 9]
        #x: grouped column in data1
        #return: mean(x) where x is filtered out all 'str' values

        #xx = x.astype(float)
        
        
        #print(xx)
        
        
        #tmp = [v for v in list(x) if isinstance(v, float)] # filter out 'str' in a grouped column
        tmp = [v for v in list(x) if v != float(1000000)] # filter out '1000' in a grouped column
        #print('\nGroupgy Date Filter out nan 1000000\n', tmp)
        #print(tmp)
        
        # what if tmp = []?
        return np.mean(tmp)
    
    tm = df.replace(to_replace='None', value= float(1000000))
    tm = df.replace(to_replace='NaN', value= float(1000000))
    tm = df.replace(to_replace='nan', value= float(1000000))

    tm['CI'] = tm.CI.astype(float)
    tm['BI'] = tm.BI.astype(float)
    tm['HI'] = tm.HI.astype(float)

    tm['Year'] = pd.to_datetime(tm['Date']).dt.year 

    tmm = tm[tm['Year'] >= 2012]

    tmm2 = tmm.groupby('Date', as_index=False).aggregate({'CI': mean_customized,
                                                         'BI': mean_customized,
                                                         'HI': mean_customized})
    return tmm2

def bi_convert_date_to_yyyy_ww(data_frame, yyyy_ww_dict, end_time = '2019-05-31'):
    '''
    Input: data_frame
    Output: data_frame added yyyy_ww column
    '''
    yyww = []
    yyyymmdd = []
    
    tmp = data_frame.Date
    date_key = list(tmp)
    
    ey, em, ed = end_time.split('-')
    
    for i in range(data_frame.shape[0]):
        k = date_key[i]
        y, m, d = k.split('/')
        if y == ey and m == '06':
            break
        kk = y + '-' + m + '-' + d
        #print('\nkk = ', kk)
        yyyymmdd.append(kk)
        yw = yyyy_ww_dict[kk]
        yyww.append(yw)

    Date2 = pd.Series(yyyymmdd)
    Year_Week = pd.Series(yyww)
    #print(Year_Week.shape)
    
    data_frame2 = data_frame.reset_index(drop = True)
    #print(data_frame2.shape)
    
    data_frame2['Year_Week'] = Year_Week
    data_frame2['Date2'] = Date2
    
    return data_frame2


import math
#x = float('nan')
#math.isnan(x)

def bi_groupby_yyyy_ww(data_frame, col = 'Year_Week'):
    
    def fill_nan_dummy(county_df):
        

        county_df2 = county_df.replace(to_replace='nan', value= float(1000000))

        county_df3 = county_df2.replace(to_replace='NaN', value= float(1000000))

        #county_df3['CI'] = county_df3['CI'].astype(float)
        
        #county_df3['BI'] = county_df3['BI'].astype(float)
        
        #county_df3['HI'] = county_df3['HI'].astype(float)
        
        return county_df3
    
    def mean_customized(x):
        #purpose: to calculate mean of a grouped column with mixed values, ex: [3, 6, 8, 'NaN', 'T', 9]
        #x: grouped column in data1
        #return: mean(x) where x is filtered out all 'str' values
        
        xx = fill_nan_dummy(x)
        #print('*'*50, type(xx))
        #tmp = [v for v in list(x) if isinstance(v, float)] # filter out 'str' in a grouped column
        tmp = [v for v in list(xx) if v != float(1000000)] # filter out '1000' in a grouped column
        
        tmp2 = [vv for vv in tmp if not math.isnan(float(vv))] # filter out 'nan'
        
       
        #print("filter out 1000000\n", tmp)
        #print('\n Filter out:\n', tmp2)
        if len(tmp2) == 0:
            print("\nXXX\n")
            return 'XXX'
        elif len(tmp2) >= 1:
            return np.mean(tmp2)


    # groupby & mean with condition & add customized function with groupby
    tmp8 = data_frame.groupby(col, as_index = False).aggregate({'BI': mean_customized,
                                                            'CI': mean_customized,
                                                            'HI': mean_customized})    

    return tmp8

def bi_gen_full_time_table(data_frame, week_total, county = ''):
    
    '''
    Input: 
        data_frame: data with missing weeks of years
        week_total: total number of yyyy_ww 
    Output: 
        full_table: data_frame fulfilled missed yyyy_ww based on week_total
    
    '''
    
    full_table = []
    j = 0
    
    for i in range(len(week_total)):

        yw_ref = week_total[i]
        
        try:
            yw_cty = sorted(data_frame.Year_Week)[j]
        except: IndexError


        if yw_ref == yw_cty:
            bi = data_frame.BI[j]
            ci = data_frame.CI[j]
            hi = data_frame.HI[j]
            full_table.append([yw_ref, hi, ci, bi])
            j += 1
            continue

        else:
            full_table.append([yw_ref,'XXX', 'XXX', 'XXX']) # float('nan'), float('nan'), float('nan')])
            continue
            
    full_table = pd.DataFrame(full_table, columns = ['YYYY_WW', 'HI', 'CI', 'BI'])
    full_table['County'] = county
    #print('*'*20, 'Full Table: ', county, '*'*20, '\n', full_table.tail())
    return full_table     

def find_missed_yyyy_ww(data_frame, week_total):
    a = set(data_frame.Year_Week)
    b = set(week_total)
    return b.difference(a)



### Utils function for Weather

In [393]:
def weather_convert_date_to_yyyy_ww(data_frame, yyyy_ww_dict):
    '''
    Input: data_frame
    Output: data_frame added yyyy_ww column
    '''
    yyww = []
    tmp = data_frame.Date
    date_key = list(tmp)
    
    for i in range(data_frame.shape[0]):
        
        k = date_key[i]
        try:
            yw = yyyy_ww_dict[k]
            #print(k, '-'*10, yw)
            yyww.append(yw)
        except: KeyError
            
        
    Year_Week = pd.Series(yyww)

    
    data_frame = data_frame.reset_index(drop = True)
    
    data_frame['Year_Week'] = Year_Week

    return data_frame




def groupby_yyyy_ww(data_frame, col = 'Year_Week'):
    
    def fill_nan_dummy(county_df):
        

        county_df = county_df.replace(to_replace='T', value=1000)
        county_df = county_df.replace(to_replace='X', value=1000)
        county_df = county_df.replace(to_replace='/', value=1000)
        county_df = county_df.replace(to_replace='...', value=1000)  

        #county_df = county_df.replace(to_replace='T', value=float('NaN'))
        #county_df = county_df.replace(to_replace='X', value=float('NaN'))
        #county_df = county_df.replace(to_replace='/', value=float('NaN'))
        #county_df = county_df.replace(to_replace='...', value=float('NaN'))
        county_df = county_df.astype(float)

        return county_df
    
    def mean_customized(x):
        #purpose: to calculate mean of a grouped column with mixed values, ex: [3, 6, 8, 'NaN', 'T', 9]
        #x: grouped column in data1
        #return: mean(x) where x is filtered out all 'str' values
        xx = fill_nan_dummy(x)
        #tmp = [v for v in list(x) if isinstance(v, float)] # filter out 'str' in a grouped column
        tmp = [v for v in list(xx) if v != 1000] # filter out '1000' in a grouped column

        
        return np.mean(tmp)


    # groupby & mean with condition & add customized function with groupby
    tmp8 = data_frame.groupby(col, as_index = False).aggregate({'Temperature': mean_customized,
                                                            'T Max': mean_customized,
                                                            'T Min': mean_customized,
                                                            'RH': mean_customized,
                                                            'Precp': mean_customized})    

    return tmp8

def add_date_to_county(folder_path, cut_year = 2012):
    county = [] # one county
    
    tmp = sorted(list(folders_files[folder_path]))
    for file in tmp:
        file_path = folder_path + file
        tmp_data = pd.read_csv(file_path)
        
        _, yyyy, mm = file.split('.')[0].split('-')
        yyyy_mm = yyyy + '-' + mm
        
        tmp2 = list(tmp_data.ObsTime)
        tmp3 = []
        for day in tmp2:
            if day < 10:
                day_str = '0'+ str(day)
            else:
                day_str = str(day)
            yyyy_mm_dd = yyyy_mm + '-' + day_str
            
            tmp3.append(yyyy_mm_dd)
            
        tmp_data['Date'] = tmp3
        
        use_cols = ['Temperature', 'T Max', 'T Min', 'RH', 'Precp', 'Date']
        
        tmp_data2 = tmp_data[use_cols]
        
        county.append(tmp_data2)
        
        county_df = pd.concat(county)
        
        # select year
        county_df['Year'] = pd.to_datetime(county_df['Date']).dt.year

        county_df = county_df[county_df['Year'] >= cut_year]
        
    return county_df
def fill_nan_dummy(county_df):
        

    #county_df = county_df.replace(to_replace='T', value=0)
    #county_df = county_df.replace(to_replace='X', value=0)
    #county_df = county_df.replace(to_replace='/', value=0)
    #county_df = county_df.replace(to_replace='...', value=0)  
    
    #county_df = county_df.replace(to_replace='T', value=float('NaN'))
    #county_df = county_df.replace(to_replace='X', value=float('NaN'))
    #county_df = county_df.replace(to_replace='/', value=float('NaN'))
    #county_df = county_df.replace(to_replace='...', value=float('NaN'))
    
    #county_df = county_df.replace(to_replace= 'inf', value=float('NaN'))

    county_df['Temperature'] = county_df['Temperature'].astype(float)
    county_df['T Max'] = county_df['T Max'].astype(float)
    county_df['T Min'] = county_df['T Min'].astype(float)
    county_df['Precp'] = county_df['Precp'].astype(float)
    county_df['RH'] = county_df['RH'].astype(float)

    
    return county_df


def fill_nan(county_df):
        

        
    county_df = county_df.replace(to_replace='T', value=float('NaN'))
    county_df = county_df.replace(to_replace='X', value=float('NaN'))
    county_df = county_df.replace(to_replace='/', value=float('NaN'))
    county_df = county_df.replace(to_replace='...', value=float('NaN'))
    #county_df = county_df.replace(to_replace= 'inf', value=float('NaN'))

    county_df['Temperature'] = county_df['Temperature'].astype(float)
    county_df['T Max'] = county_df['T Max'].astype(float)
    county_df['T Min'] = county_df['T Min'].astype(float)
    county_df['Precp'] = county_df['Precp'].astype(float)
    county_df['RH'] = county_df['RH'].astype(float)

    #data_x2 = pd.DataFrame(data_x, columns = weather_cols, dtype = float)

    #df['feature_count'].fillna(df['client_name'].map(s), inplace=True)
    county_df['Temperature'] = county_df['Temperature'].fillna(county_df['Temperature'].mean())
    
    county_df['T Max'] = county_df['T Max'].fillna(county_df['T Max'].mean())
    county_df['T Min'] = county_df['T Min'].fillna(county_df['T Min'].mean())
    county_df['Precp'] = county_df['Precp'].fillna(county_df['Precp'].mean()) 
    county_df['RH'] = county_df['RH'].fillna(county_df['RH'].mean()) 
    
    return county_df

def weather_convert_date_to_yyyy_ww(data_frame, yyyy_ww_dict):
    '''
    Input: data_frame
    Output: data_frame added yyyy_ww column
    '''
    yyww = []
    tmp = data_frame.Date
    date_key = list(tmp)
    
    for i in range(data_frame.shape[0]):
        
        k = date_key[i]
        try:
            yw = yyyy_ww_dict[k]
            #print(k, '-'*10, yw)
            yyww.append(yw)
        except: KeyError
            
        
    Year_Week = pd.Series(yyww)

    
    data_frame = data_frame.reset_index(drop = True)
    
    data_frame['Year_Week'] = Year_Week

    return data_frame

def weather_gen_full_time_table(data_frame, week_total, county = ''):
    '''
    Input: 
        data_frame: data with missing weeks of years
        week_total: total number of yyyy_ww 
    Output: 
        full_table: data_frame fulfilled missed yyyy_ww based on week_total
    
    '''
    
    full_table = []
    j = 0
    for i in tqdm(range(len(week_total))):
        #print(i)

        yw_ref = week_total[i]
        
        try:
            yw_cty = sorted(data_frame.Year_Week)[j]
        except: IndexError
            
        #print(yw_cty)
        #print(j, yw_ref, yw_cty)

        if yw_ref == yw_cty:
            temp = data_frame['Temperature'][j]
            tmax = data_frame['T Max'][j]
            tmin = data_frame['T Min'][j]
            rh = data_frame['RH'][j]
            recp = data_frame['Precp'][j]

            full_table.append([yw_ref, temp, tmax, tmin, rh, recp])
            j += 1
            continue

        else:
            full_table.append([yw_ref, float('nan'), float('nan'), float('nan')])
            continue
            
    full_table = pd.DataFrame(full_table, columns = ['YYYY_WW', 'Temperature', 'T Max', 'T Min', 'RH', 'Precp'])
    full_table['County'] = county
    
    return full_table   

# TW county in Chinese English dict
cty_chinese = ['台北市', '高雄市', '台中市', '宜蘭縣', '台南市', 
               '新北市', '彰化縣', '花蓮縣', '雲林縣', 
               '新竹縣', '嘉義縣', '桃園市', '澎湖縣', '南投縣', 
               '屏東縣', '台東縣', '基隆市', '苗栗縣'] # '連江縣', '金門縣'

cty_english = ['Taipei', 'Kaohsiung', 'Taichung', 'Yilan',  'Tainan',
               'New Taipei', 'Changhua', 'Hualien', 'Yunlin',
               'Hsinchu',  'Chiayi',  'Taoyuan',  'Penghu',  'Nantou',
               'Pingtung', 'Taitung',  'Keelung', 'Miaoli']

cty_cn_en = collections.defaultdict()
cty_en_cn = collections.defaultdict()

# create 2 dict
for cn, en in zip(cty_chinese, cty_english):
    cty_cn_en[cn] = en
    cty_en_cn[en] = cn


## Generate YYYY_WW data standard

In [335]:
from datetime import date,timedelta

df = pd.DataFrame({'INSP_DATE2':pd.date_range(start = '2012-01-01', end = '2019-5-31')})

a =  df['INSP_DATE2'] - pd.offsets.Week(weekday=6)
b =  df['INSP_DATE2'] + pd.offsets.Week(weekday=5)

m1 = df['INSP_DATE2'] != (a + pd.offsets.Week())
m2 = df['INSP_DATE2'] != (b - pd.offsets.Week())

df['c'] = df['INSP_DATE2'].mask(m1, a)
df['d'] = df['INSP_DATE2'].mask(m2, b)

rows = []
w = 1 # start week
y = 2012 # start year

# generate standard yyyy_ww

for b in tqdm(range(0, 2755, 7)):

    row = []
    win = df.iloc[b : (b+7), :]
    date = win.c
    #print(str(date).split('-'))
    #y, _, _ = str(list(date)[0]).split('-')
    
       
    if w <10:
        w_str = '0' + str(w)
    else:
        w_str = str(w)
    win["Year_Week"] = str(y) + '_' + str(w_str)
    
    if y != 2014 and w == 52:
        w = 0
        y += 1
    elif y == 2014 and w == 53:
        w = 0
        y += 1
    w += 1
    
    rows.append(win)
    
# cancat  
year_weekly = pd.concat(rows)

# unique week
week_total = year_weekly.Year_Week.unique()

# yyyy_mm_dd: yyyy_ww
date_week = collections.defaultdict()
for i in range(year_weekly.shape[0]):
    date_key = year_weekly.INSP_DATE2[i]
    k = str(date_key).split(' ')[0]
    week_val = year_weekly.Year_Week[i]
    date_week[k] = week_val

  0%|          | 0/394 [00:00<?, ?it/s]/home/basic/tensorflow/tensorflow_pkg/tf_1.12_cuda10.0/lib/python3.5/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 394/394 [00:28<00:00, 13.99it/s]


### Start pre-process BI

In [559]:
# BI data

# replace 'None' with 'NaN' & groupby 'Date'

data_fin2 = {}


print('\n Processing! \n')


for k in data_fin.keys():    

    df = data_fin[k]
    
    tmm = bi_groupby_date(df)    

    tm2 = bi_convert_date_to_yyyy_ww(tmm, date_week, end_time = '2019-05-31')  
    
    tm2['Date2'] = pd.to_datetime(tm2['Date2'])  

    tm2 = tm2[(tm2['Date2'] <= '2019-05-31')]
    
    #print(k, '\n', tm2.tail(50), '\n')

    tm3 = bi_groupby_yyyy_ww(tm2)
    
    #print(k, '\n', tm3.tail(), '\n')
    
    tm4 = bi_gen_full_time_table(tm3, week_total, county = k)

    print(k, '\n', tm4.head(20), '\n')

    data_fin2[k] = tm4
    



 Processing! 


XXX

花蓮縣 
     YYYY_WW         HI        CI         BI County
0   2012_01        0.5   6.24975        0.5    花蓮縣
1   2012_02          0         0          0    花蓮縣
2   2012_03          0         0          0    花蓮縣
3   2012_04        XXX       XXX        XXX    花蓮縣
4   2012_05          0         0          0    花蓮縣
5   2012_06          0         0          0    花蓮縣
6   2012_07          0         0          0    花蓮縣
7   2012_08  0.0571429   0.19044  0.0571429    花蓮縣
8   2012_09          0         0          0    花蓮縣
9   2012_10          0         0          0    花蓮縣
10  2012_11  0.0666667      0.16  0.0666667    花蓮縣
11  2012_12          0         0          0    花蓮縣
12  2012_13          0         0          0    花蓮縣
13  2012_14          0         0          0    花蓮縣
14  2012_15   0.133333  0.333333   0.133333    花蓮縣
15  2012_16       0.18  0.430533       0.18    花蓮縣
16  2012_17        0.5   1.38875        0.5    花蓮縣
17  2012_18          0         0          0    花蓮縣
18 


XXX


XXX


XXX


XXX

基隆市 
     YYYY_WW         HI        CI         BI County
0   2012_01          1    1.8515          1    基隆市
1   2012_02   0.166667     0.641   0.166667    基隆市
2   2012_03          0         0          0    基隆市
3   2012_04          2      12.5          2    基隆市
4   2012_05   0.666667   1.14933   0.666667    基隆市
5   2012_06          0         0          0    基隆市
6   2012_07       0.25   0.89275       0.25    基隆市
7   2012_08  0.0666667    0.2564  0.0666667    基隆市
8   2012_09        XXX       XXX        XXX    基隆市
9   2012_10          0         0          0    基隆市
10  2012_11       0.25     9.375       0.75    基隆市
11  2012_12        0.5     1.226        0.5    基隆市
12  2012_13   0.133333  0.277733   0.133333    基隆市
13  2012_14          0         0          0    基隆市
14  2012_15   0.666667   4.76167   0.666667    基隆市
15  2012_16        0.5     3.625       0.75    基隆市
16  2012_17       0.55    5.1923       0.55    基隆市
17  2012_18   0.666667   1.85167   0.666667    基隆市
1

彰化縣 
     YYYY_WW         HI        CI         BI County
0   2012_01   0.361111   1.63682   0.361111    彰化縣
1   2012_02   0.622222   3.34674   0.622222    彰化縣
2   2012_03       0.24   0.72222       0.28    彰化縣
3   2012_04          0         0          0    彰化縣
4   2012_05  0.0833333  0.231458  0.0833333    彰化縣
5   2012_06   0.461818     1.109   0.461818    彰化縣
6   2012_07    2.23333   5.59013    2.23333    彰化縣
7   2012_08   0.758333   3.09869   0.858333    彰化縣
8   2012_09    1.46667   2.91353    1.46667    彰化縣
9   2012_10   0.754286   3.51631   0.754286    彰化縣
10  2012_11   0.764286   2.08678   0.926786    彰化縣
11  2012_12      0.325  0.824437      0.325    彰化縣
12  2012_13        0.5    1.4705        0.5    彰化縣
13  2012_14          2   6.77325        2.4    彰化縣
14  2012_15   0.993333   3.47745   0.993333    彰化縣
15  2012_16    1.64064   6.70629    1.64064    彰化縣
16  2012_17   0.580952   1.95338   0.580952    彰化縣
17  2012_18    2.28571    5.3228    2.80952    彰化縣
18  2012_19       1.82   

In [552]:
# Final step for BI, CI, HI
data_fin2['宜蘭縣'].tail()

,YYYY_WW,HI,CI,BI,County
382,2019_18,0.151174,0.347208,0.080125,宜蘭縣
383,2019_19,0.349941,13.8067,0.349941,宜蘭縣
384,2019_20,0.25825,2.02366,0.25825,宜蘭縣
385,2019_21,0.353208,9.84647,0.419875,宜蘭縣
386,2019_22,0.5592,4.23562,0.4592,宜蘭縣


# Pre-processing Weather data

In [63]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [64]:
mypath = './data/tw_weekly_weather/'
listdir(mypath)

['Miaoli',
 'Tainan',
 'Penghu',
 'Chiayi',
 'Taichung',
 'Hsinchu',
 'Nantou',
 'Pingtung',
 'Changhua',
 'Lienchang',
 'Taoyuan',
 'Taitung',
 'Kinmen',
 'New Taipei',
 'Yunlin',
 'Hualien',
 'Keelung',
 'Yilan',
 'Kaohsiung',
 'Taipei']

In [181]:
folders = [mypath + folder + '/' for folder in listdir(mypath)]
print(folders)

['./data/tw_weekly_weather/Miaoli/', './data/tw_weekly_weather/Tainan/', './data/tw_weekly_weather/Penghu/', './data/tw_weekly_weather/Chiayi/', './data/tw_weekly_weather/Taichung/', './data/tw_weekly_weather/Hsinchu/', './data/tw_weekly_weather/Nantou/', './data/tw_weekly_weather/Pingtung/', './data/tw_weekly_weather/Changhua/', './data/tw_weekly_weather/Lienchang/', './data/tw_weekly_weather/Taoyuan/', './data/tw_weekly_weather/Taitung/', './data/tw_weekly_weather/Kinmen/', './data/tw_weekly_weather/New Taipei/', './data/tw_weekly_weather/Yunlin/', './data/tw_weekly_weather/Hualien/', './data/tw_weekly_weather/Keelung/', './data/tw_weekly_weather/Yilan/', './data/tw_weekly_weather/Kaohsiung/', './data/tw_weekly_weather/Taipei/']


In [182]:
from collections import defaultdict

folders_files = defaultdict(list)
for folder in folders:
    folders_files[folder] = listdir(folder)

### Get final preprocessed for Weather


In [299]:

all_county_weather = collections.defaultdict()

print('\n Count missing weeks of weather \n')

for key_county, value_files in folders_files.items():
    
    cty_name = key_county.split('/')[-2]
    if cty_name == 'Kinmen' or cty_name == 'Lienchang':
        continue
    
    county = add_date_to_county(key_county, cut_year = 2012)
    
    cty1 = weather_convert_date_to_yyyy_ww(county, date_week)
    
    print('\n', '*' * 20, 'Cty name = ', cty_name, '*'*20)

    # groupby week
    cty2 = groupby_yyyy_ww(cty1, col = 'Year_Week')
    
    #print('\n', cty2.tail())    
    
    all_county_weather[cty_name] = cty2


 Count missing weeks of weather 


 ******************** Cty name =  Taichung ********************

     Year_Week  Temperature         RH      T Max      T Min      Precp
382   2019_18    24.885714  78.285714  29.242857  22.100000  18.428571
383   2019_19    23.414286  85.571429  27.414286  20.771429  10.000000
384   2019_20    27.657143  80.714286  31.671429  24.742857  12.357143
385   2019_21    25.400000  84.142857  29.485714  22.914286  36.666667
386   2019_22    26.850000  78.500000  31.250000  23.433333   3.083333

 ******************** Cty name =  Yilan ********************

     Year_Week  Temperature         RH      T Max      T Min      Precp
382   2019_18    23.285714  83.285714  27.071429  20.585714  12.700000
383   2019_19    21.757143  87.714286  24.714286  19.700000  17.066667
384   2019_20    26.200000  87.428571  30.614286  23.657143  20.871429
385   2019_21    24.285714  80.000000  27.614286  21.642857   3.314286
386   2019_22    25.000000  81.833333  29.433333  22.


 ******************** Cty name =  Tainan ********************

     Year_Week  Temperature         RH      T Max      T Min      Precp
382   2019_18    26.071429  73.285714  29.485714  23.400000   7.142857
383   2019_19    24.571429  78.714286  28.171429  21.928571   3.642857
384   2019_20    29.057143  71.285714  32.528571  26.628571   3.500000
385   2019_21    27.514286  75.000000  30.900000  24.885714  10.250000
386   2019_22    28.150000  72.666667  31.750000  25.066667   1.666667


In [300]:
# example: Miaoli weather
all_county_weather['Taichung'].tail()

,Year_Week,Temperature,RH,T Max,T Min,Precp
382,2019_18,24.885714,78.285714,29.242857,22.100000,18.428571
383,2019_19,23.414286,85.571429,27.414286,20.771429,10.000000
384,2019_20,27.657143,80.714286,31.671429,24.742857,12.357143
385,2019_21,25.400000,84.142857,29.485714,22.914286,36.666667
386,2019_22,26.850000,78.500000,31.250000,23.433333,3.083333


In [501]:
compiled = []
final_weather_dict = {}
for cty, cty_df in all_county_weather.items():
    cty_df['County_EN'] = cty
    cty_df2 = cty_df[['County_EN', 'Year_Week', 'Temperature', 'RH', 'T Max', 'T Min', 'Precp']]
    compiled.append(cty_df2)
    final_weather_dict[cty] = cty_df2
final_weather = pd.concat(compiled, axis = 0, ignore_index = True)

In [502]:
save_path = './data/final_weather.csv'
final_weather.to_csv(save_path)

In [503]:
# example: Miaoli BI, CI, HI
data_fin2[cty_en_cn['Miaoli']].tail()

,YYYY_WW,HI,CI,BI,County
382,2019_18,2.5735,13.1403,2.99017,苗栗縣
383,2019_19,3,26.19,4,苗栗縣
384,2019_20,3.16667,15.838,3,苗栗縣
385,2019_21,2.73333,13.6534,3.4,苗栗縣
386,2019_22,2.52255,11.3397,2.92255,苗栗縣


### Compile Weather + BI data

In [553]:
full_data = []
for cty_cn, cty_en in cty_cn_en.items():
    bi = data_fin2[cty_cn]
    te = final_weather_dict[cty_en]
    tmp = [bi, te]
    cty_bi_te = pd.concat(tmp, axis = 1) # concat column-by-column & keep old column names
    full_data.append(cty_bi_te)
    

In [554]:
full_data2 = pd.concat(full_data, axis = 0, ignore_index = True) # concat row-by-row & auto generate new index

In [555]:
full_data2.shape

(6966, 12)

# FINAL PREPROCESSED DATA (FULL 18 COUNTY + WEATHER + BI, CI, HI)

In [556]:
full_data2.head()

,YYYY_WW,HI,CI,BI,County,County_EN,Year_Week,Temperature,RH,T Max,T Min,Precp
0,2012_01,0.5,6.24975,0.5,花蓮縣,Hualien,2012_01,16.614286,75.714286,18.657143,14.714286,0.571429
1,2012_02,0,0,0,花蓮縣,Hualien,2012_02,18.942857,75.428571,21.557143,16.800000,0.357143
2,2012_03,0,0,0,花蓮縣,Hualien,2012_03,19.300000,76.428571,21.914286,17.100000,2.400000
3,2012_04,XXX,XXX,XXX,花蓮縣,Hualien,2012_04,16.142857,79.714286,18.600000,13.771429,5.000000
4,2012_05,0,0,0,花蓮縣,Hualien,2012_05,17.957143,76.571429,20.414286,16.057143,1.000000


In [558]:
write_data = full_data2[['County_EN', 'YYYY_WW', 'BI', 'CI', 'HI', 'Temperature', 'T Max', 'T Min', 'RH', 'Precp']]
write_data.head()
write_data.to_csv('./data/full_house_again.csv')

##### END